In [1]:
# 安裝yahoo finance套件
!pip install yfinance

In [21]:
import google.generativeai as genai  # Google生成式AI套件Gemini API
import os  # 系統檔，用來操作檔案與文件
import requests  # 用來發HTTP請求，用於網路爬蟲
from bs4 import BeautifulSoup  # 網路爬蟲套件
import numpy as np  # 數值計算
import pandas as pd  # 資料處理
import yfinance as yf  # Yahoo finance 股價資訊
import datetime as dt  # 處理日期時間
import time  # 時間延遲套件，用於控制API呼叫頻率


# 爬取股票資訊的類別
class StockInfo():
    """
    股票基本資訊爬取類別

    功能：
    1. 從台灣證券交易所爬取所有上市公司的股號、股名、產業別
    2. 提供股號查詢股名的功能

    資料來源：台灣證券交易所 (TWSE) ISIN網站
    """

    # 取得全部股票的股號、股名
    def stock_name(self):
        """
        爬取所有台股的股號、股名、產業別

        Returns:
            DataFrame: 包含「股號」、「股名」、「產業別」三欄的資料表

        技術說明：
        - 使用requests取得網頁內容
        - 使用BeautifulSoup解析HTML
        - 過濾出4碼股號的上市公司（排除ETF等其他商品）
        """
        # 向台灣證券交易所發送請求
        response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
        # 使用BeautifulSoup解析HTML內容
        url_data = BeautifulSoup(response.text, 'html.parser')
        # 找出所有的表格列（tr標籤）
        stock_company = url_data.find_all('tr')

        # 資料處理：提取股號、股名、產業別
        # \u3000 是全形空白字元，用來分隔股號和股名
        # 只保留4碼的股號（過濾掉ETF、特別股等）
        data = [
            (row.find_all('td')[0].text.split('\u3000')[0].strip(),
                row.find_all('td')[0].text.split('\u3000')[1],
                row.find_all('td')[4].text.strip())
            for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
        ]
        # 建立DataFrame並設定欄位名稱
        df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])
        return df

    # 取得股票名稱
    def get_stock_name(self, stock_id, name_df):
        """
        根據股號查詢股名

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            name_df (DataFrame): 包含股號和股名的資料表

        Returns:
            str: 股票名稱（例如：'台積電'）
        """
        return name_df.set_index('股號').loc[stock_id, '股名']

In [22]:
class StockAnalysis():
    """
    股票分析類別 - 整合股價資料與Gemini AI分析

    主要功能：
    1. 取得股票價格資料（技術面）
    2. 取得股票基本面資料（營收、EPS）
    3. 使用Gemini AI進行股票分析
    4. 多檔股票比較與推薦

    設計理念：
    - 整合多元數據源（價格、基本面）
    - 利用LLM進行智能分析
    """

    def __init__(self, gemini_api_key):
        """
        初始化股票分析系統

        Args:
            gemini_api_key (str): Google Gemini API金鑰

        初始化步驟：
        1. 設定Gemini API
        2. 建立生成模型實例
        """
        # 設定Gemini API金鑰
        genai.configure(api_key=gemini_api_key)
        """
        1. Gemini 2.5 Pro - 進階思考模型
        2. Gemini 2.5 Flash - 快速且智慧
        3. Gemini 2.5 Flash-Lite - 速度最快的 Flash 模型，可提高成本效益和輸送量。
        """
        # 使用Gemini 2.5 Pro模型（高階推理能力）
        self.model = genai.GenerativeModel('gemini-2.5-pro')
        # 建立股票資訊物件
        self.stock_info = StockInfo()
        # 預先載入所有股票名稱（避免重複爬取）
        self.name_df = self.stock_info.stock_name()


    def stock_price(self, stock_id, days=15):
        """
        取得股票近期價格資料（技術面分析基礎）

        Args:
            stock_id (str): 股票代號（例如：'2330'）
            days (int): 要取得的天數（預設15天）

        Returns:
            dict: 包含日期、收盤價、每日報酬率的字典

        技術說明：
        - 使用yfinance API取得Yahoo Finance資料
        - 計算每日報酬率 = (今日收盤-昨日收盤)/昨日收盤
        - 自動加上.TW後綴（台股代碼格式）
        """

        # 台股代碼需要加上.TW後綴
        stock_id += '.TW'
        # 計算日期區間
        end = dt.date.today() # 今天
        start = end - dt.timedelta(days=days) # 往前推N天

        # 禁用 FutureWarning
        import warnings
        warnings.filterwarnings('ignore', category=FutureWarning)

        # 下載股價資料
        # auto_adjust=True 會自動調整除權息的價格
        df = yf.download(stock_id, start=start, auto_adjust=True)

        # 將欄位名稱改為中文（方便閱讀）
        df.columns = ['開盤價', '最高價', '最低價', '收盤價', '成交量']
        # 建立回傳的資料結構
        data = {
            '日期': df.index.strftime('%Y-%m-%d').tolist(), # 格式化日期
            '收盤價': df['收盤價'].tolist(), # 收盤價列表
            '每日報酬': df['收盤價'].pct_change().tolist() # 報酬率 = 價格變化百分比
        }
        return data


    def stock_fundamental(self, stock_id):
        """
        取得股票基本面資料（價值投資分析基礎）

        Args:
            stock_id (str): 股票代號

        Returns:
            dict: 包含季度日期、營收成長率、EPS、EPS季增率

        基本面指標說明：
        - 營收成長率：季度營收相比上一季的成長百分比
        - EPS (每股盈餘)：公司獲利能力的核心指標
        - EPS季增率：EPS相比上一季的成長百分比

        注意事項：
        - 某些股票可能無基本面資料（新上市或資料不全）
        - 使用try-except確保程式不會因單一股票錯誤而中斷
        """

        stock_id += '.TW'
        stock = yf.Ticker(stock_id)

        try:
            # 計算季度營收成長率
            # pct_change(-1)：與前一期（-1）比較的百分比變化
            # fill_method=None：不填補缺失值
            quarterly_revenue_growth = np.round(
                stock.quarterly_financials.loc['Total Revenue'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得季度EPS
            quarterly_eps = np.round(
                stock.quarterly_financials.loc['Basic EPS'].dropna().tolist(),
                2
            )
            # 計算EPS季增率
            quarterly_eps_growth = np.round(
                stock.quarterly_financials.loc['Basic EPS'].pct_change(-1, fill_method=None).dropna().tolist(),
                2
            )
            # 取得日期並格式化
            dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]
            # 組合資料（確保長度一致）
            data = {
                '季日期': dates[:len(quarterly_revenue_growth)],
                '營收成長率': quarterly_revenue_growth.tolist(),
                'EPS': quarterly_eps.tolist(),
                'EPS 季增率': quarterly_eps_growth.tolist()
            }
        except Exception as e:
            # 如果取得資料失敗，回傳空資料
            print(f"無法取得 {stock_id} 基本面資料: {e}")
            data = {'季日期': [], '營收成長率': [], 'EPS': [], 'EPS 季增率': []}

        return data

    def _get_reply(self, content_msg):
        """
        呼叫Gemini API並處理各種回應狀況（內部方法）

        Args:
            content_msg (str): 要傳送給Gemini的提示詞（prompt）

        Returns:
            str: Gemini的回應文字或錯誤訊息

        錯誤處理機制：
        1. finish_reason = 1 (STOP)：正常完成
        2. finish_reason = 2 (MAX_TOKENS)：超過長度限制
        3. finish_reason = 3 (SAFETY)：觸發安全政策
        4. finish_reason = 4 (RECITATION)：引用問題
        5. HTTP 429：API速率限制（免費版：每分鐘5次，每天25次）
        6. HTTP 503：服務暫時不可用
        """

        try:
            # 呼叫Gemini API生成內容
            response = self.model.generate_content(
                content_msg,
                generation_config=genai.types.GenerationConfig(
                    max_output_tokens=10000,  # 最大輸出token數（避免過長）
                    temperature=1.0,  # 溫度參數（1.0 = 較有創意，0.0 = 較保守）
                )
            )

            # 檢查是否有回應候選
            if not response.candidates:
                return "模型未返回任何回應"

            candidate = response.candidates[0]
            finish_reason = candidate.finish_reason

            # finish_reason是整數值，代表不同的完成狀態：
            # 0 = UNSPECIFIED (未指定)
            # 1 = STOP (正常完成)
            # 2 = MAX_TOKENS (達到token限制)
            # 3 = SAFETY (安全過濾)
            # 4 = RECITATION (引用問題)
            # 5 = OTHER (其他原因)

            # 根據不同的完成原因處理
            if finish_reason == 1:  # STOP - 正常完成
                # 正常完成
                return candidate.content.parts[0].text

            elif finish_reason == 2:  # MAX_TOKENS - 達到長度限制
                try:
                    partial_text = candidate.content.parts[0].text if candidate.content.parts else ""
                    return f"{partial_text}\n\n [回應被截斷，已達到長度限制]"
                except:
                    return "回應超過長度限制。建議：1)增加 max_output_tokens 2)簡化 prompt"

            elif finish_reason == 3:  # SAFETY - 安全政策
                return "模型因安全政策無法提供回應"

            elif finish_reason == 4:  # RECITATION - 引用問題
                return "回應因引用問題被過濾"

            else:
                return f"未知完成原因: {finish_reason}"

        except Exception as e:
            # 處理API錯誤
            error_msg = str(e)
            # 速率限制錯誤（最常見）
            if "429" in error_msg or "RESOURCE_EXHAUSTED" in error_msg:
                return "達到 API 速率限制（免費層：5次/分鐘，25次/天）"
            # 服務暫時不可用
            elif "503" in error_msg:
                return "服務暫時不可用，請稍後重試"
            # 其他錯誤
            else:
                return f"API 錯誤: {error_msg}"


    def stock_gimini_analysis(self, stock_id):
        """
        使用Gemini AI分析單支股票（核心功能）

        Args:
            stock_id (str): 股票代號

        Returns:
            str: Gemini生成的股票分析報告

        分析流程：
        1. 取得股票名稱
        2. 收集技術面資料（價格、報酬率）
        3. 收集基本面資料（營收、EPS）
        4. 將資料摘要後傳送給Gemini
        5. 讓Gemini扮演金融分析師角色，產生專業分析

        Prompt設計要點：
        - 角色設定：明確告訴模型扮演「金融分析師」
        - 數據提供：精簡但完整的關鍵資訊
        - 輸出要求：技術面+基本面+風險評估
        - 語言指定：繁體中文
        """
        # 取得股票名稱
        stock_name = self.stock_info.get_stock_name(stock_id, self.name_df)

        # 收集股價資料
        price_data = self.stock_price(stock_id)
        print('資料收集完畢')
        # 收集基本面資料
        stock_value_data = self.stock_fundamental(stock_id)

        # --- 資料摘要製作 ---
        # 目的：將大量資料濃縮成關鍵指標，避免超過token限制
        summary_parts = []

        # 股價摘要：最新價格 + 兩週漲跌幅
        if price_data and price_data['收盤價']:
            latest = price_data['收盤價'][-1] # 最新收盤價
            first = price_data['收盤價'][0] # 首日收盤價
            change = ((latest - first) / first) * 100 # 計算漲跌百分比
            summary_parts.append(f"股價：{latest:.2f} (兩週 {change:+.2f}%)")

        # 基本面摘要：最新一季的營收成長和EPS
        if stock_value_data and stock_value_data['季日期']:
            q = stock_value_data['季日期'][0] # 季度
            rev = stock_value_data['營收成長率'][0] # 營收成長率
            eps = stock_value_data['EPS'][0] # EPS
            summary_parts.append(f"基本面({q})：營收成長{rev*100:.1f}%, EPS {eps:.2f}")

        # 組合所有摘要
        combined = "；".join(summary_parts)

        # ============ 作業修改區 START ============
        # --- 建構Prompt ---
        # 這是Prompt Engineering的關鍵：如何引導LLM產生專業分析
        """
        'conservative': '''你是保守型財務顧問...
        1. 優先評估下檔風險
        2. 關注股息穩定性
        3. 避免高波動股票...''',

        'aggressive': '''你是成長型投資顧問...
        1. 關注營收成長動能
        2. 尋找突破訊號...''',

        'balanced': '''你是專業金融分析師...
        平衡評估機會與風險...'''
        """
        content_msg = f'''你是一位具備CFA等級分析能力的投資研究員，原則：
1. 報告需包含 EPS、毛利率、ROE、自由現金流等財務指標。
2. 同時分析 Top-down（全球/產業）與 Bottom-up（公司）。
3. 評估合理估值（PE、PB、EV/EBITDA）。
4. 給予「中性＋數據化」建議，而非感覺派。{stock_name}。
資料：{combined}

    請提供詳細的技術面和基本面見解，並考慮風險因素。繁體中文回應。'''
        # ============ 作業修改區 END ============
        # 呼叫Gemini API
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_sort(self, message):
        """
        讓Gemini為多檔股票評分並排序

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini生成的評分排序結果

        評分標準：
        - 50分為基準
        - 正面因素加分（例如：營收成長、技術面轉強）
        - 負面因素扣分（例如：虧損、下跌趨勢）
        - 綜合技術面+基本面+市場環境

        應用場景：投資組合篩選、資金配置決策
        """

        content_msg = f'''你是專業股票分析師，根據趨勢報告評分(0-100)。
50分為基準，正面消息加分，負面消息扣分。
排序所有股票。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply


    def stock_gimini_choice(self, message):
        """
        讓Gemini從多檔股票中選出最佳投資標的

        Args:
            message (dict): 包含多檔股票分析報告的字典

        Returns:
            str: Gemini選出的最佳股票及理由

        決策邏輯：
        - 即使所有選項都不理想，也要選出相對最佳的
        - 需要說明選擇理由（風險、報酬、時機等）
        - 模擬真實的投資決策情境

        教學價值：
        - 理解LLM如何進行多維度評估
        - 學習投資決策的思考框架
        """

        content_msg = f'''你是專業證券分析師，從趨勢報告中選出最適合投資的一檔股票。
即使都不理想也要選一檔，說明理由。{str(message)}(繁體中文)'''
        reply = self._get_reply(content_msg)
        return reply

In [23]:
# 【重要】請修改為你自己的Gemini API Key
# ============ 作業修改區 START ============
gemini_api_key = 'API Key# ============ 作業修改區 END ============
# 建立股票分析物件
stock_analysis = StockAnalysis(gemini_api_key)

# 單支股價分析

In [24]:
# ============ 作業修改區 START ============
# 改成想分析的股票代號
reply = stock_analysis.stock_gimini_analysis('0050')
# ============ 作業修改區 END ============
print(reply)

[*********************100%***********************]  1 of 1 completed


資料收集完畢
無法取得 0050.TW 基本面資料: 'Total Revenue'
好的，收到您的指示。我將以CFA等級的投資研究員身份，為您提供一份關於元大台灣50 (0050.TW) 的詳細投資分析報告。

***

### **元大台灣卓越50基金 (0050.TW) 投資分析報告**

**報告日期：** 2024年5月23日
**分析師：** [您的CFA級分析助理]

**報告起始，請注意：** 您提供的股價61.55元與目前市價(約172元新台幣)有較大差距，本報告將以2024年5月23日市場價格約 **172.00元新台幣** 為分析基礎。您提及的「兩週 -2.22%」回檔數據，將納入近期市場情緒與技術面分析。

---

### **1. 報告摘要 (Executive Summary)**

元大台灣50 (0050) 是追蹤臺灣50指數的被動式ETF，為投資者提供了對台灣市值最大、流動性最佳的50家公司的單一投資管道。

*   **基本面：** 0050的價值高度集中於以台積電為首的電子產業，直接受益於全球AI、HPC等長期科技趨勢。其綜合財務指標（加權平均）顯示出高品質的獲利能力(高ROE)與穩健的現金流，但估值已來到歷史相對高檔。
*   **技術面：** 長期處於上升趨勢，但短期因漲多而出現技術性回檔，呈現高檔震盪格局。關鍵支撐與壓力位值得關注。
*   **估值：** 目前本益比(P/E)約21.5倍，高於五年歷史均值約17倍，顯示市場已反映部分樂觀預期。
*   **建議：** **中性。** 鑒於強勁的產業趨勢與偏高的估值並存，建議採取中性觀點。長期投資者可考慮在市場出現非基本面因素修正、回測關鍵支撐區時分批佈局。短期交易者則需留意波動加劇的風險。

---

### **2. 由上而下分析 (Top-Down Analysis)**

#### **2.1 全球宏觀環境**

*   **利率政策：** 美國聯準會(Fed)維持高利率環境以對抗通膨，市場對降息時點的預期不斷延後。高利率環境對企業融資成本與終端消費需求構成壓力，但對金融業的利差有正面貢獻。
*   **AI與半導體週期：** 全球AI伺服器與數據中心建置需求強勁，成為半導體產業最主要的成長動能。消費性電子（PC、手機）雖有復甦跡象，但力道尚待觀察。此趨勢對台灣以

# 收集多檔股票的趨勢報告

In [25]:
# 建立股票清單
# ============ 作業修改區 START ============
# 請修改為你想分析的股票清單（建議3-5檔，避免超過API限制）
# 台股股票參考：
# 0050 前十大持股參考：
# 2330 台積電、2317 鴻海、2454 聯發科、2308 台達電、3711 日月光投控
# 2382 廣達、2881 富邦金、2891 中信金、2882 國泰金、2303 聯電
stock_list = ['2330', '2454', '2308']
# ============ 作業修改區 END ============

# 設定儲存路徑
today_time = dt.date.today().strftime('%Y%m%d')
path = './StockGemini/TrendReport/'
os.makedirs(path, exist_ok=True)

# 建立多檔股票的趨勢報告並儲存
content = {}

print(f"\n{'='*70}")
print(f"開始分析 {len(stock_list)} 檔股票")
print(f"{'='*70}\n")

# 批次處理每一檔股票
for i, stock in enumerate(stock_list, 1):
    print(f"[{i}/{len(stock_list)}] 處理股票: {stock}")

    file_path = f"{path}trend_{stock}_{today_time}.txt"

    if not os.path.exists(file_path):
        try:
            # 分析股票
            print(f"   正在分析...")
            analysis = stock_analysis.stock_gimini_analysis(stock_id=stock)

            # 儲存結果
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(analysis)

            content[stock] = analysis
            print(f"   分析完成並已儲存\n")

            # 速率限制控制
            if i < len(stock_list):
                wait_time = 32
                print(f"   等待 {wait_time} 秒以符合 API 速率限制...\n")
                time.sleep(wait_time)

        except Exception as e:
            error_msg = f"錯誤: {str(e)}"
            print(f"   {error_msg}\n")
            content[stock] = error_msg

    else:
        # 讀取已存在的分析
        with open(file_path, "r", encoding="utf-8") as f:
            content[stock] = f.read()
        print(f"   從快取讀取\n")

print(f"{'='*70}")
print("所有股票分析完成！")
print(f"{'='*70}\n")

print("\n" + "="*70)
print("分析結果")
print("="*70 + "\n")

for stock_id, analysis in content.items():
    try:
        stock_name = stock_analysis.stock_info.get_stock_name(stock_id, stock_analysis.name_df)
    except KeyError:
        stock_name = stock_id # Fallback to using stock_id as name if not found
        print(f"警告: 股票代號 {stock_id} 的名稱未在資料中找到。")

    print("="*70)
    print(f"【{stock_id} - {stock_name}】")
    print("="*70)
    print(analysis)
    print("\n")


開始分析 3 檔股票

[1/3] 處理股票: 2330
   從快取讀取

[2/3] 處理股票: 2454
   從快取讀取

[3/3] 處理股票: 2308
   正在分析...


[*********************100%***********************]  1 of 1 completed


資料收集完畢
   分析完成並已儲存

所有股票分析完成！


分析結果

【2330 - 台積電】
好的，謹遵指示。以下為您提供一份針對台積電（TSMC）的CFA等級投資研究報告。

---

### **台積電 (2330.TW) 投資研究報告**

**報告日期：** 2024年6月12日
**分析師：** [您的姓名/職稱]
**目標公司：** 台灣積體電路製造股份有限公司 (TSMC)
**目前股價 (假設)：** 1,450.00 新台幣
**評級：** **中性 (Neutral)**

---

### **1. 摘要 (Executive Summary)**

台積電憑藉其無可取代的先進製程技術領導地位，在全球半導體產業中佔據核心戰略位置。AI、高效能運算 (HPC) 的結構性需求為公司提供了強勁的長期成長動能。然而，以目前1,450元的股價評估，多數樂觀預期已反映在股價中，估值處於歷史相對高檔。地緣政治風險、資本支出壓力及產業週期性波動是潛在的下行風險。

綜合評估，雖然公司基本面無虞且長期展望正向，但目前的風險回報比 (Risk/Reward Ratio) 趨於平衡。因此，我們給予**「中性」**評級，並建議投資人關注更具吸引力的切入點或採用分批佈局策略。

---

### **2. 投資論點分析**

#### **2.1 Top-down 分析 (全球與產業)**

*   **全球宏觀趨勢：AI驅動的半導體新紀元**
    *   **人工智慧 (AI) 需求爆發：** AI是驅動半導體產業成長的最主要引擎。從NVIDIA的GPU、AMD的MI系列晶片，到各大雲端服務商 (CSP) 自行開發的ASIC晶片，幾乎所有高階AI晶片皆依賴台積電的先進製程 (N5/N4、N3) 及CoWoS先進封裝技術。此趨勢並非短期現象，而是長達數年的結構性轉變，為台積電提供了清晰且強勁的需求能見度。
    *   **終端應用復甦：** 智慧型手機、個人電腦 (PC) 等消費性電子產品市場已觸底反彈，雖然復甦力道溫和，但庫存去化已近尾聲，預期將為成熟製程與部分先進製程帶來穩定需求。

*   **半導體產業週期與展望**
    *   **上升週期確立：** 全球半導體產業正處於新一輪上升週期的開端。根據世界半導體貿易統計組織 (WSTS) 預測，

In [26]:
reply = stock_analysis.stock_gimini_choice(str(content))
print(reply)

好的，身為專業證券分析師，我已詳細研讀您提供的三份趨勢報告。我的任務是，即使都不完美，也要從中選出「相對最適合」投資的一檔股票，並闡明我的決策邏輯。

綜合評估三份報告，我會選擇 **2330 台積電 (TSMC)** 作為最適合的投資標的。

以下是我的詳細分析與決策過程：

### **總結：為何選擇台積電 (2330)**

我的選擇基於「**成長確定性**」與「**產業定價權**」這兩個核心原則。在當前的市場環境下，投資的首要目標是尋找最清晰、最不易被顛覆的長期趨勢。台積電在AI時代的「軍火商」地位，使其成長路徑的能見度遠高於另外兩家公司。雖然其估值不低，但相較於台達電的極高估值和聯發科的業務不確定性，台積電的「合理溢價」是三者中風險回報比最佳的選擇。

---

### **三檔個股的比較分析 (Comparative Analysis)**

我將從三個關鍵維度來比較這三家公司：

#### **1. 成長確定性與護城河 (Growth Certainty & Moat)**

*   **台積電 (2330):** **最高。** 其護城河是三者中最深的。在先進製程領域，台積電擁有近乎壟斷的地位 (>90% 市佔率)。AI是未來數年最確定的科技主軸，而所有高階AI晶片都必須依賴台積電。這意味著它的成長與整個AI產業的擴張直接掛鉤，確定性極高。
*   **台達電 (2308):** **次高。** 受益於AI伺服器電源和散熱的強勁需求，這部分成長動能明確。但報告也指出，其電動車和工業自動化業務面臨短期逆風。這意味著台達電的成長是由「一個火熱的引擎」和「兩個溫吞的引擎」共同驅動，整體成長速度可能被部分抵銷。
*   **聯發科 (2454):** **最低。** 雖然AI手機是個誘人的題材，但智慧型手機市場本質上是消費性電子，具有高度的景氣循環性與不確定性。報告中特別提到，2025年Q3的預估營收年減-6.0%，這是一個巨大的警訊，暗示其成長路徑可能不如預期平穩。此外，它在高階市場與高通的競爭極其激烈，護城河相對較淺。

**結論：在成長的確定性上，台積電 > 台達電 > 聯發科。**

#### **2. 估值與安全邊際 (Valuation & Margin of Safety)**

這是一個「三高」的比較：股價高、估值高、市場期望高。

*   

# 推薦股票的評分排序

In [27]:
reply = stock_analysis.stock_gimini_sort(str(content))
print(reply)

好的，身為專業股票分析師，我已詳細研讀您提供的三份投資研究報告。我將根據報告中的正面與負面資訊，以50分為基準進行評分，並提供最終排序。

### **趨勢報告評分與排序**

| **排名** | **股票代號** | **公司名稱** | **趨勢評分** | **摘要** |
| :--- | :--- | :--- | :---: | :--- |
| **1** | **2454** | **聯發科** | **67** | 估值相對合理，處於分析師評估區間下緣，提供較佳的風險回報比。AI手機趨勢為長期主要動能，但短期需留意競爭與終端需求雜音。 |
| **2** | **2330** | **台積電** | **65** | 基本面與產業地位無懈可擊，AI需求確立長期成長。主要扣分項為估值已反映多數利多，安全邊際有限，且地緣政治為潛在變數。 |
| **3** | **2308** | **台達電** | **55** | 擁抱AI與綠能雙重強勁題材，成長性極佳。但估值極度昂貴，遠超歷史區間，股價已嚴重透支未來預期，高檔風險顯著。 |

---

### **個股評分解析**

以下是每支股票的詳細評分依據：

#### **1. 聯發科 (2454) - 趨勢評分: 67**

*   **基準分：50**
*   **正面加分 (+22):**
    *   **AI手機結構性趨勢 (+15):** 旗艦級天璣晶片具備AI競爭力，是推升產品單價(ASP)與毛利率的核心驅動力，成長故事清晰。
    *   **估值具吸引力 (+7):** 目前股價(1,180元)落在報告評估的合理價值區間(1,125元至1,440元)的下緣，提供了相對較高的安全邊際。
*   **負面扣分 (-5):**
    *   **短期逆風與競爭 (-5):** 報告點出短期股價回檔、技術面偏弱、與高通競爭激烈，以及對2025年Q3營收年減數據的擔憂，顯示短期不確定性較高。

**綜合評論：** 聯發科的投資價值在於其**「合理的價格」**。儘管短期面臨挑戰，但其估值尚未像其他AI概念股一樣極度昂貴，為投資人提供了較佳的風險回報平衡。若AI手機銷量能兌現，股價上檔潛力可期。

#### **2. 台積電 (2330) - 趨勢評分: 65**

*   **基準分：50